In [1]:
import sys
from pathlib import Path

src_path = Path.cwd().parent / "src"
sys.path.append(str(src_path))

import json
import types
from collections import OrderedDict
from pprint import pprint

import torch
import torch.nn as nn
from dataset import get_new_dataloader
from funcs import get_dataloaders
from models import PreActResNet18, WideResNet
from train import epoch, epoch_test


In [2]:
# constant
CWD_PATH = Path.cwd()
MODEL_DIR_PATH = CWD_PATH.parent / "models/CIFAR10-diff-norm-value-cossim-grayscale"
SETTINGS = [
    {
        "normalization_mean": [-0.631, -0.648, 0.624],
        "normalization_std": None,
    },
    {
        "normalization_mean": [-0.631, -0.97, -0.058],
        "normalization_std": None,
    },
    {
        "normalization_mean": [-0.668, -0.004, 0.165],
        "normalization_std": None,
    },
    {
        "normalization_mean": [0.26, 0.855, -0.536],
        "normalization_std": None,
    },
    {
        "normalization_mean": [0.365, -0.892, -0.559],
        "normalization_std": None,
    },
    {
        "normalization_mean": [0.456, 0.837, 0.251],
        "normalization_std": None,
    },
    {
        "normalization_mean": [0.598, 0.036, -0.537],
        "normalization_std": None,
    },
    {
        "normalization_mean": [0.648, -0.572, 0.483],
        "normalization_std": None,
    },
    {
        "normalization_mean": [0.78, 0.026, -0.51],
        "normalization_std": None,
    },
    {
        "normalization_mean": [0.847, -0.447, 0.64],
        "normalization_std": None,
    },
]


In [4]:
for setting in SETTINGS:
    normalization_mean = setting['normalization_mean']
    normalization_std = setting['normalization_std']
    if normalization_mean != None and normalization_std != None:
        mean_str = str(normalization_mean).replace(' ', '').replace(',', '_')[1:-1]
        std_str = str(normalization_std).replace(' ', '').replace(',', '_')[1:-1]
        label = f"model_rgb-grayscale_mean_{mean_str}_std_{std_str}"
    elif normalization_mean != None:
        mean_str = str(normalization_mean).replace(' ', '').replace(',', '_')[1:-1]
        label = f"model_rgb-grayscale_mean_{mean_str}"
    elif normalization_std != None:
        std_str = str(normalization_std).replace(' ', '').replace(',', '_')[1:-1]
        label = f"model_rgb-grayscale_std_{std_str}"
    else:
        label = f"model__baseline"

    print(label)

model_rgb-grayscale_mean_-0.631_-0.648_0.624
model_rgb-grayscale_mean_-0.631_-0.97_-0.058
model_rgb-grayscale_mean_-0.668_-0.004_0.165
model_rgb-grayscale_mean_0.26_0.855_-0.536
model_rgb-grayscale_mean_0.365_-0.892_-0.559
model_rgb-grayscale_mean_0.456_0.837_0.251
model_rgb-grayscale_mean_0.598_0.036_-0.537
model_rgb-grayscale_mean_0.648_-0.572_0.483
model_rgb-grayscale_mean_0.78_0.026_-0.51
model_rgb-grayscale_mean_0.847_-0.447_0.64


In [6]:
def load_model(
    model_type: str,
    location: str,
    model_normalize: bool,
):
    device = torch.device("cuda:{0}".format(0) if torch.cuda.is_available() else "cpu")

    # load model CIFAR10
    if model_type == "pre-act-18":
        model = PreActResNet18(num_classes=10, normalize=model_normalize)
    elif model_type == "independent":
        model = WideResNet(
            n_classes=10,
            depth=16,  # deep_full for CIFAR10
            widen_factor=1,
            normalize=model_normalize,
            dropRate=0.3,
        )
    else:
        model = WideResNet(
            n_classes=10,
            depth=28,  # deep_full for CIFAR10
            widen_factor=10,
            normalize=model_normalize,
            dropRate=0.3,
        )

    try:
        model = model.to(device)
        model.load_state_dict(torch.load(location, map_location=device))
    except:
        model = nn.DataParallel(model).to(device)
        model.load_state_dict(torch.load(location, map_location=device))
    model.eval()
    return model


In [9]:
model_name__tr_acc = OrderedDict()

# model performance in TR
for setting in SETTINGS:
    normalization_mean = setting['normalization_mean']
    normalization_std = setting['normalization_std']
    if normalization_mean != None and normalization_std != None:
        mean_str = str(normalization_mean).replace(' ', '').replace(',', '_')[1:-1]
        std_str = str(normalization_std).replace(' ', '').replace(',', '_')[1:-1]
        label = f"model_rgb-grayscale_mean_{mean_str}_std_{std_str}"
    elif normalization_mean != None:
        mean_str = str(normalization_mean).replace(' ', '').replace(',', '_')[1:-1]
        label = f"model_rgb-grayscale_mean_{mean_str}"
    elif normalization_std != None:
        std_str = str(normalization_std).replace(' ', '').replace(',', '_')[1:-1]
        label = f"model_rgb-grayscale_std_{std_str}"
    else:
        label = f"model__baseline"

    model_path = MODEL_DIR_PATH / label / "final.pt"
    print(model_path)
    model = load_model(
        model_type="teacher",
        location=str(model_path),
        model_normalize=False,
    )

    args = types.SimpleNamespace()
    device = torch.device("cuda:{0}".format(0) if torch.cuda.is_available() else "cpu")
    args.device = device
    args.mode = "teacher"
    args.experiment = 'diff-norm-value'
    args.dataset = 'CIFAR10'
    args.batch_size = 1000
    args.num_workers = 8
    args.normalization_mean = normalization_mean
    args.normalization_std = normalization_std
    args.extra_preprocessing_type = "rgb-grayscale"
    train_loader, test_loader = get_new_dataloader(args=args)

    _, train_acc = epoch(
        args,
        train_loader,
        model,
        teacher=None,
        lr_schedule=None,
        epoch_i=None,
        opt=None,
    )
    _, test_acc = epoch_test(
        args,
        test_loader,
        model,
    )
    print(train_acc)
    print(test_acc)

    model_name__tr_acc[label] = {
        "train_acc": train_acc,
        "test_acc": test_acc,
    }


/data/weijing/DI-ting_normalization/models/CIFAR10-diff-norm-value-cossim-grayscale/model_rgb-grayscale_mean_-0.631_-0.648_0.624/final.pt
Custom_normalization: Normalize(mean=[-0.631, -0.648, 0.624], std=(0.2471, 0.2435, 0.2616))
Grayscale(num_output_channels=3)
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:23<00:00,  2.16it/s]


1.0
0.9288
/data/weijing/DI-ting_normalization/models/CIFAR10-diff-norm-value-cossim-grayscale/model_rgb-grayscale_mean_-0.631_-0.97_-0.058/final.pt
Custom_normalization: Normalize(mean=[-0.631, -0.97, -0.058], std=(0.2471, 0.2435, 0.2616))
Grayscale(num_output_channels=3)
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:23<00:00,  2.14it/s]


1.0
0.9291
/data/weijing/DI-ting_normalization/models/CIFAR10-diff-norm-value-cossim-grayscale/model_rgb-grayscale_mean_-0.668_-0.004_0.165/final.pt
Custom_normalization: Normalize(mean=[-0.668, -0.004, 0.165], std=(0.2471, 0.2435, 0.2616))
Grayscale(num_output_channels=3)
Files already downloaded and verified
Files already downloaded and verified


...

0.9999
0.9289
/data/weijing/DI-ting_normalization/models/CIFAR10-diff-norm-value-cossim-grayscale/model_rgb-grayscale_mean_0.78_0.026_-0.51/final.pt
Custom_normalization: Normalize(mean=[0.78, 0.026, -0.51], std=(0.2471, 0.2435, 0.2616))
Grayscale(num_output_channels=3)
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:23<00:00,  2.13it/s]


1.0
0.9263
/data/weijing/DI-ting_normalization/models/CIFAR10-diff-norm-value-cossim-grayscale/model_rgb-grayscale_mean_0.847_-0.447_0.64/final.pt
Custom_normalization: Normalize(mean=[0.847, -0.447, 0.64], std=(0.2471, 0.2435, 0.2616))
Grayscale(num_output_channels=3)
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:23<00:00,  2.12it/s]


1.0
0.9252


In [10]:
pprint(model_name__tr_acc)

# Serializing json
json_object = json.dumps(model_name__tr_acc, indent=2)
 
# Writing to sample.json
with open("tr_performance.json", "w") as outfile:
    outfile.write(json_object)

OrderedDict([('model_rgb-grayscale_mean_-0.631_-0.648_0.624',
              {'test_acc': 0.9288, 'train_acc': 1.0}),
             ('model_rgb-grayscale_mean_-0.631_-0.97_-0.058',
              {'test_acc': 0.9291, 'train_acc': 1.0}),
             ('model_rgb-grayscale_mean_-0.668_-0.004_0.165',
              {'test_acc': 0.9267, 'train_acc': 0.99998}),
             ('model_rgb-grayscale_mean_0.26_0.855_-0.536',
              {'test_acc': 0.9275, 'train_acc': 0.99996}),
             ('model_rgb-grayscale_mean_0.365_-0.892_-0.559',
              {'test_acc': 0.9265, 'train_acc': 0.99996}),
             ('model_rgb-grayscale_mean_0.456_0.837_0.251',
              {'test_acc': 0.9305, 'train_acc': 0.99996}),
             ('model_rgb-grayscale_mean_0.598_0.036_-0.537',
              {'test_acc': 0.9282, 'train_acc': 0.99996}),
             ('model_rgb-grayscale_mean_0.648_-0.572_0.483',
              {'test_acc': 0.9289, 'train_acc': 0.9999}),
             ('model_rgb-grayscale_mean_0.78_0.

In [7]:
model_name__te_acc = OrderedDict()

# model performance in FE
for setting in SETTINGS:
    normalization_mean = setting['normalization_mean']
    normalization_std = setting['normalization_std']
    if normalization_mean != None and normalization_std != None:
        mean_str = str(normalization_mean).replace(' ', '').replace(',', '_')[1:-1]
        std_str = str(normalization_std).replace(' ', '').replace(',', '_')[1:-1]
        label = f"model_rgb-grayscale_mean_{mean_str}_std_{std_str}"
    elif normalization_mean != None:
        mean_str = str(normalization_mean).replace(' ', '').replace(',', '_')[1:-1]
        label = f"model_rgb-grayscale_mean_{mean_str}"
    elif normalization_std != None:
        std_str = str(normalization_std).replace(' ', '').replace(',', '_')[1:-1]
        label = f"model_rgb-grayscale_std_{std_str}"
    else:
        label = f"model__baseline"

    model_path = MODEL_DIR_PATH / label / "final.pt"
    print(model_path)
    model = load_model(
        model_type="teacher",
        location=str(model_path),
        model_normalize=False,
    )

    train_loader, test_loader = get_dataloaders(
        dataset="CIFAR10",
        batch_size=1000,
        pseudo_labels=False,
        normalize=True,
        train_shuffle=False,
    )

    args = types.SimpleNamespace()
    device = torch.device("cuda:{0}".format(0) if torch.cuda.is_available() else "cpu")
    args.device = device
    args.mode = "teacher"

    _, train_acc = epoch(
        args,
        train_loader,
        model,
        teacher=None,
        lr_schedule=None,
        epoch_i=None,
        opt=None,
    )
    _, test_acc = epoch_test(
        args,
        test_loader,
        model,
    )
    print(train_acc)
    print(test_acc)

    model_name__te_acc[label] = {
        "train_acc": train_acc,
        "test_acc": test_acc,
    }



/data/weijing/DI-ting_normalization/models/CIFAR10-diff-norm-value-cossim-grayscale/model_rgb-grayscale_mean_-0.631_-0.648_0.624/final.pt
No Transform
Files already downloaded and verified
Files already downloaded and verified
No Transform
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:35<00:00,  1.39it/s]


0.10308
0.1053
/data/weijing/DI-ting_normalization/models/CIFAR10-diff-norm-value-cossim-grayscale/model_rgb-grayscale_mean_-0.631_-0.97_-0.058/final.pt
No Transform
Files already downloaded and verified
Files already downloaded and verified
No Transform
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:31<00:00,  1.61it/s]


0.12208
0.125
/data/weijing/DI-ting_normalization/models/CIFAR10-diff-norm-value-cossim-grayscale/model_rgb-grayscale_mean_-0.668_-0.004_0.165/final.pt
No Transform
Files already downloaded and verified
Files already downloaded and verified
No Transform
Files already downloaded and verified
Files already downloaded and verified


...

0.3427
0.3372
/data/weijing/DI-ting_normalization/models/CIFAR10-diff-norm-value-cossim-grayscale/model_rgb-grayscale_mean_0.78_0.026_-0.51/final.pt
No Transform
Files already downloaded and verified
Files already downloaded and verified
No Transform
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:31<00:00,  1.60it/s]


0.42368
0.4044
/data/weijing/DI-ting_normalization/models/CIFAR10-diff-norm-value-cossim-grayscale/model_rgb-grayscale_mean_0.847_-0.447_0.64/final.pt
No Transform
Files already downloaded and verified
Files already downloaded and verified
No Transform
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:31<00:00,  1.60it/s]


0.5377
0.5044


In [8]:
pprint(model_name__te_acc)

# Serializing json
json_object = json.dumps(model_name__te_acc, indent=2)
 
# Writing to sample.json
with open("fe_performance.json", "w") as outfile:
    outfile.write(json_object)

OrderedDict([('model_rgb-grayscale_mean_-0.631_-0.648_0.624',
              {'test_acc': 0.1053, 'train_acc': 0.10308}),
             ('model_rgb-grayscale_mean_-0.631_-0.97_-0.058',
              {'test_acc': 0.125, 'train_acc': 0.12208}),
             ('model_rgb-grayscale_mean_-0.668_-0.004_0.165',
              {'test_acc': 0.1077, 'train_acc': 0.10872}),
             ('model_rgb-grayscale_mean_0.26_0.855_-0.536',
              {'test_acc': 0.225, 'train_acc': 0.2304}),
             ('model_rgb-grayscale_mean_0.365_-0.892_-0.559',
              {'test_acc': 0.1062, 'train_acc': 0.10558}),
             ('model_rgb-grayscale_mean_0.456_0.837_0.251',
              {'test_acc': 0.7737, 'train_acc': 0.84172}),
             ('model_rgb-grayscale_mean_0.598_0.036_-0.537',
              {'test_acc': 0.3097, 'train_acc': 0.32378}),
             ('model_rgb-grayscale_mean_0.648_-0.572_0.483',
              {'test_acc': 0.3372, 'train_acc': 0.3427}),
             ('model_rgb-grayscale_mean_0.